##  differential expression gene

In [2]:
import pandas as pd
import numpy as np
import math

In [ ]:
#read the data
data = pd.read_csv("All_genes.csv")
data = data.drop([0,1,2])
data.columns = ['Tracking_ID','BRH704545_C1','BRH692283_C2','BRH712959_C3','BRH721340_C4','BRH721344_C5',
                'N068_C6','N048_C8','N090_C9','N098_C10','SB09_A1','SB46_A2','SB60_A3','SB72_A4',
                'SB79_A5','L027_A6','L072_A7','L078_A8','L140_A9','L102_A10','L115_A11','L133_A12',
                'L137_A13','L149_A14','L074_A15']
data = data.set_index('Tracking_ID')
data_T_cell = data[['N068_C6','N048_C8','N090_C9','N098_C10','L027_A6','L072_A7',
                    'L078_A8','L140_A9','L102_A10','L115_A11','L133_A12','L137_A13',
                    'L149_A14','L074_A15']].astype(str).astype(float)
data_Whole_Blood = data[['BRH704545_C1','BRH692283_C2','BRH712959_C3','BRH721340_C4',
                         'BRH721344_C5','SB09_A1','SB46_A2','SB60_A3','SB72_A4','SB79_A5']].astype(str).astype(float)

data_T_cell.head(2)

hign_IR_gene = pd.read_csv("CD4_high_IR_gene_list.txt")
hign_IR_gene_list = hign_IR_gene['GPN2'].values.tolist()
hign_IR_gene_list.append('GPN2')

In [3]:
data.head()

,BRH704545_C1,BRH692283_C2,BRH712959_C3,BRH721340_C4,BRH721344_C5,N068_C6,N048_C8,N090_C9,N098_C10,SB09_A1,...,L027_A6,L072_A7,L078_A8,L140_A9,L102_A10,L115_A11,L133_A12,L137_A13,L149_A14,L074_A15
Tracking_ID,,,,,,,,,,,,,,,,,,,,,
A1BG,2.40382,2.43021,2.22905,1.8493,1.9584,4.96318,6.10044,4.35813,5.02215,1.52474,...,4.58245,5.84451,3.29078,5.80016,6.98869,14.0711,1.62981,6.19747,1.70015,3.00811
A1BG-AS1,1.37407,1.08803,1.27321,1.63272,1.13689,0.745182,0.364248,0.700179,0.540295,1.63288,...,0.504396,0.519882,0.395447,1.63584,0.606268,0.510604,0.225992,0.596801,0.630004,0.388694
A1CF,0,0,0,0,0,0,0,0.00826804,0.00161386,0,...,0.0036656,0,0,0.00450583,0,0,0.00940826,0,0,0
A2M,0.0531876,0.118585,0.0848329,0.161905,0.0329815,0.00995046,0.0576083,0.00672003,0.00984416,0.0571031,...,0.0149112,0.0406488,0.0431386,0.0396378,0.0107209,0.00769042,0.101929,0,0.0337985,0.0332221
A2M-AS1,0.29666,0.422854,0.174637,0.518284,0.245445,0.48023,0.143089,0.112146,0.189595,0.21246,...,0.0747574,0.294029,0.152093,0.182653,0.477597,0.0940974,0.0991737,0.339683,0.261294,0.118479


In [50]:
t_cell_list = []
for i in list(range(0,14)):
    c = data_T_cell[data_T_cell.columns[i]].values
    t_cell_list.append(c)
t_cell = np.asarray(t_cell_list) 

whole_blood_list =[]
for i in list(range(0,10)):
    b = data_Whole_Blood[data_Whole_Blood.columns[i]].values
    whole_blood_list.append(b)
whole_blood = np.asarray(whole_blood_list)

t_cell_labels = data_T_cell.columns.get_values()
whole_blood_labels = data_Whole_Blood.columns.get_values()

t_cell

array([[  4.96318000e+00,   7.45182000e-01,   0.00000000e+00, ...,
          1.03024000e+02,   1.38688000e+01,   9.30725000e+00],
       [  6.10044000e+00,   3.64248000e-01,   0.00000000e+00, ...,
          7.43528000e+01,   1.38810000e+01,   9.03538000e+00],
       [  4.35813000e+00,   7.00179000e-01,   8.26804000e-03, ...,
          5.85407000e+01,   1.70919000e+01,   1.14912000e+01],
       ..., 
       [  6.19747000e+00,   5.96801000e-01,   0.00000000e+00, ...,
          9.72665000e+01,   1.20437000e+01,   1.26081000e+01],
       [  1.70015000e+00,   6.30004000e-01,   0.00000000e+00, ...,
          7.96356000e+01,   1.58478000e+01,   9.70776000e+00],
       [  3.00811000e+00,   3.88694000e-01,   0.00000000e+00, ...,
          1.20427000e+02,   1.05759000e+01,   9.31639000e+00]])

In [51]:
# normalize the data
from sklearn.preprocessing import Normalizer
scaler = Normalizer().fit(t_cell)
t_cell = scaler.transform(t_cell)

scaler = Normalizer().fit(whole_blood)
whole_blood = scaler.transform(whole_blood)

data_t_cell = pd.DataFrame(t_cell)
data_T_cell = data_t_cell.transpose()
data_T_cell.columns = t_cell_labels
data_T_cell.index = data.index

data_whole_blood = pd.DataFrame(whole_blood)
data_Whole_Blood = data_whole_blood.transpose()
data_Whole_Blood.columns = whole_blood_labels
data_Whole_Blood.index = data.index

In [52]:
data_T_cell['T_Control_Avg'] = (data_T_cell['N068_C6']
                                +data_T_cell['N048_C8']
                                +data_T_cell['N090_C9']
                                +data_T_cell['N098_C10'])/4
data_T_cell['T_Active_Avg'] = (data_T_cell['L027_A6']
                                +data_T_cell['L072_A7']
                                +data_T_cell['L078_A8']
                                +data_T_cell['L140_A9']
                                +data_T_cell['L102_A10']
                                +data_T_cell['L115_A11']
                                +data_T_cell['L133_A12']
                                +data_T_cell['L137_A13']
                                +data_T_cell['L149_A14']
                                +data_T_cell['L074_A15'])/10
data_T_cell['T_Active/Control']  = data_T_cell['T_Active_Avg']/data_T_cell['T_Control_Avg']
data_T_cell['T_Active-Control']  = data_T_cell['T_Active_Avg']-data_T_cell['T_Control_Avg']

data_T_cell_Diff = data_T_cell[['T_Active_Avg','T_Control_Avg','T_Active/Control','T_Active-Control']]
data_T_cell_Diff.sort_values(by='T_Active/Control',ascending=False)

,T_Active_Avg,T_Control_Avg,T_Active/Control,T_Active-Control
Tracking_ID,,,,
DGAT2L6,1.809791e-07,0.0,inf,1.809791e-07
CERS3-AS1,2.667813e-07,0.0,inf,2.667813e-07
FRMPD2B,1.585859e-06,0.0,inf,1.585859e-06
ASPA,1.583875e-07,0.0,inf,1.583875e-07
FRMPD2,1.776331e-07,0.0,inf,1.776331e-07
FRG2EP,3.602339e-08,0.0,inf,3.602339e-08
FRG2B,2.419842e-07,0.0,inf,2.419842e-07
LOC643355,9.520685e-08,0.0,inf,9.520685e-08
SMIM9,1.024249e-07,0.0,inf,1.024249e-07


In [76]:
data_T_cell = data_T_cell.transpose()
data_T_cell

Tracking_ID,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
N068_C6,4.96318,0.745182,0.000000,0.009950,0.480230,0.000000,0.029018,0.000000,0.000000,0.000000,...,2.66917,2.33942,1.83393,5.65296,12.74850,0.013328,4.96498,103.0240,13.86880,9.30725
N048_C8,6.10044,0.364248,0.000000,0.057608,0.143089,0.000000,0.103114,0.000000,0.023710,0.000000,...,1.98762,4.21423,4.62936,14.18130,16.76450,0.040824,4.74138,74.3528,13.88100,9.03538
N090_C9,4.35813,0.700179,0.008268,0.006720,0.112146,0.000000,0.073276,0.017728,0.000000,0.000000,...,3.05461,2.73463,4.64947,13.29070,15.55380,0.044081,5.45840,58.5407,17.09190,11.49120
N098_C10,5.02215,0.540295,0.001614,0.009844,0.189595,0.000000,0.071310,0.017236,0.000000,0.000000,...,2.97998,3.58354,4.31326,11.95790,16.96140,0.021961,4.84225,48.2365,17.81250,11.18110
L027_A6,4.58245,0.504396,0.003666,0.014911,0.074757,0.000000,0.016322,0.000000,0.052313,0.000000,...,3.33925,4.32394,1.94827,6.82021,11.62730,0.004993,5.26327,104.3130,13.96700,10.87700
L072_A7,5.84451,0.519882,0.000000,0.040649,0.294029,0.006047,0.014785,0.000000,0.000000,0.000000,...,3.47705,8.17782,3.29074,10.49360,13.98340,0.033330,5.43006,122.0060,11.41980,10.78570
L078_A8,3.29078,0.395447,0.000000,0.043139,0.152093,0.044547,0.047134,0.000000,0.025206,0.000000,...,4.80220,10.88430,2.60551,5.79446,9.78328,0.007861,5.17781,199.0520,12.24120,6.89823
L140_A9,5.80016,1.635840,0.004506,0.039638,0.182653,0.000000,0.065971,0.015927,0.000000,0.000000,...,2.99292,2.95261,3.49517,11.05890,15.88340,0.028556,4.46426,78.7505,17.08120,10.90500
L102_A10,6.98869,0.606268,0.000000,0.010721,0.477597,0.000000,0.000000,0.000000,0.025065,0.010039,...,3.56573,6.48112,3.38232,9.14042,11.11150,0.023933,5.00207,106.4300,9.44394,10.02760
L115_A11,14.07110,0.510604,0.000000,0.007690,0.094097,0.000000,0.016762,0.000000,0.008978,0.000000,...,3.45899,2.46981,3.88971,8.73440,9.66423,0.000000,3.38292,87.0763,8.19684,7.52513


In [77]:
hign_IR_gene_list.remove('C11orf83')
hign_IR_gene_list.remove('LOC100506469')
hign_IR_gene_list.remove('MTERFD3')
hign_IR_gene_list.remove('C14orf64')
hign_IR_gene_list.remove('LOC100133445')

data_T_cell_overlap = data_T_cell[hign_IR_gene_list]
data_T_cell_overlap = data_T_cell_overlap.transpose()
data_T_cell_overlap

,N068_C6,N048_C8,N090_C9,N098_C10,L027_A6,L072_A7,L078_A8,L140_A9,L102_A10,L115_A11,L133_A12,L137_A13,L149_A14,L074_A15
Tracking_ID,,,,,,,,,,,,,,
SPIN2B,4.723410,4.971830,2.915890,3.376640,4.244150,4.370150,4.045900,5.400500,4.059580,6.945050,4.234460,5.707730,4.131320,4.402590
COA5,9.546480,5.782750,7.463700,6.437220,8.875520,8.975590,6.375040,12.455300,8.634720,10.095400,9.253650,8.505730,10.604000,8.531650
SGK494,0.894441,0.627184,0.981042,0.656223,0.208480,0.568199,0.116145,2.267900,0.173198,0.052550,0.257214,0.680836,1.708370,0.161034
CENPM,2.754280,5.311600,4.187690,4.057770,3.904510,9.121300,9.503730,2.023990,5.714820,4.557250,29.651900,6.221700,5.988950,9.672310
MIS12,11.960500,20.160700,18.976000,21.862200,11.667200,13.475800,8.055170,12.447400,11.630600,12.262400,11.239100,15.418600,12.769200,11.456200
UBAP1L,0.874558,1.026730,0.871414,0.860745,0.166250,0.577628,0.186768,1.863360,0.252140,0.142402,0.341839,1.198060,1.245170,0.148265
DGCR14,3.001620,4.617270,4.424440,5.751710,2.815990,2.428170,2.490240,4.385960,2.527650,2.796140,3.633320,3.061310,2.834540,2.503800
TAGLN,2.746850,4.754180,3.200420,3.910720,1.461730,2.323070,12.442600,4.608250,0.911385,1.519480,4.836650,2.426980,5.199170,1.243930
ZNF668,3.738560,4.254430,3.889260,4.421000,3.411740,4.361400,3.527700,3.656170,3.220850,3.400340,4.136780,3.818600,3.169980,3.246760


In [80]:
data_T_cell_overlap['T_Control_Avg'] = (data_T_cell_overlap['N068_C6']
                                +data_T_cell_overlap['N048_C8']
                                +data_T_cell_overlap['N090_C9']
                                +data_T_cell_overlap['N098_C10'])/4
data_T_cell_overlap['T_Active_Avg'] = (data_T_cell_overlap['L027_A6']
                                +data_T_cell_overlap['L072_A7']
                                +data_T_cell_overlap['L078_A8']
                                +data_T_cell_overlap['L140_A9']
                                +data_T_cell_overlap['L102_A10']
                                +data_T_cell_overlap['L115_A11']
                                +data_T_cell_overlap['L133_A12']
                                +data_T_cell_overlap['L137_A13']
                                +data_T_cell_overlap['L149_A14']
                                +data_T_cell_overlap['L074_A15'])/10
data_T_cell_overlap['T_Active/Control']  = data_T_cell_overlap['T_Active_Avg']/data_T_cell_overlap['T_Control_Avg']
data_T_cell_overlap['T_Active-Control']  = data_T_cell_overlap['T_Active_Avg']-data_T_cell_overlap['T_Control_Avg']

data_T_cell_overlap_Diff = data_T_cell_overlap[['T_Active_Avg','T_Control_Avg','T_Active/Control','T_Active-Control']]
data_T_cell_overlap_Diff.sort_values(by='T_Active/Control',ascending=False).head(30)

,T_Active_Avg,T_Control_Avg,T_Active/Control,T_Active-Control
Tracking_ID,,,,
CLDND2,0.576080,0.172517,3.339262,0.403563
DOC2GP,0.552864,0.216130,2.558023,0.336735
PSMA5,23.029990,9.478158,2.429796,13.551833
CENPM,8.636046,4.077835,2.117802,4.558211
LRRC61,7.007066,3.378395,2.074081,3.628671
PPM1N,3.060701,1.590435,1.924443,1.470266
TXNDC17,4.856989,2.661720,1.824756,2.195269
OSGEP,14.026634,8.232953,1.703719,5.793681
SLC35E2,9.617856,6.024100,1.596563,3.593756


------------------------------------------------------------

In [9]:
data_Whole_Blood['WB_Control_Avg'] = (data_Whole_Blood['BRH704545_C1']
                                +data_Whole_Blood['BRH692283_C2']
                                +data_Whole_Blood['BRH712959_C3']
                                +data_Whole_Blood['BRH721340_C4']
                                 +data_Whole_Blood['BRH721344_C5'])/5
data_Whole_Blood['WB_Active_Avg'] = (data_Whole_Blood['SB09_A1']
                                +data_Whole_Blood['SB46_A2']
                                +data_Whole_Blood['SB60_A3']
                                +data_Whole_Blood['SB72_A4']
                                +data_Whole_Blood['SB79_A5'])/5
data_Whole_Blood['WB_Active/Control']  = data_Whole_Blood['WB_Active_Avg']/data_Whole_Blood['WB_Control_Avg'] 
data_Whole_Blood['WB_Active-Control']  = data_Whole_Blood['WB_Active_Avg']-data_Whole_Blood['WB_Control_Avg']

data_Whole_Blood_Diff = data_Whole_Blood[['WB_Active_Avg','WB_Control_Avg','WB_Active/Control','WB_Active-Control']]
data_Whole_Blood_Diff = data_Whole_Blood_Diff.sort_values(by='WB_Active/Control',ascending=False)
data_Whole_Blood_Diff.head(100)

,WB_Active_Avg,WB_Control_Avg,WB_Active/Control,WB_Active-Control
Tracking_ID,,,,
LOC51145,1.014619e-06,0.0,inf,1.014619e-06
CXADRP3,2.472730e-07,0.0,inf,2.472730e-07
ROPN1B,6.360622e-07,0.0,inf,6.360622e-07
LPP-AS1,8.734726e-07,0.0,inf,8.734726e-07
C6orf223,2.776338e-07,0.0,inf,2.776338e-07
CYP3A43,2.222805e-07,0.0,inf,2.222805e-07
C7orf69,5.682140e-07,0.0,inf,5.682140e-07
TTC23L,2.327007e-07,0.0,inf,2.327007e-07
C8B,2.758825e-07,0.0,inf,2.758825e-07


In [51]:
data_Whole_Blood_Diff_list = data_Whole_Blood_Diff.head(100).index.tolist()

In [27]:
overlap_whole_blood_high_IR_gene_list = []
for a in hign_IR_list:
    for b in data_Whole_Blood_Diff.index.values.tolist():
        if a == b:
            overlap_whole_blood_high_IR_gene_list.append(a) 
overlap_whole_blood_high_IR_gene_list

['GPN2']

------------------------------------------------------------

## Random Forest 

### Random forest for t-cell

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split

In [37]:
data_T_cell = data_T_cell.drop(['T_Control_Avg','T_Active_Avg','T_Active/Control','T_Active-Control'],1)
data_T_cell = data_T_cell.transpose()
data_T_cell

Tracking_ID,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
N068_C6,0.000122,0.000018,0.000000e+00,2.446738e-07,0.000012,0.000000e+00,7.135366e-07,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000066,0.000058,0.000045,0.000139,0.000313,3.277174e-07,0.000122,0.002533,0.000341,0.000229
N048_C8,0.000126,0.000008,0.000000e+00,1.190847e-06,0.000003,0.000000e+00,2.131516e-06,0.000000e+00,4.901098e-07,0.000000e+00,...,0.000041,0.000087,0.000096,0.000293,0.000347,8.438955e-07,0.000098,0.001537,0.000287,0.000187
N090_C9,0.000139,0.000022,2.628885e-07,2.136683e-07,0.000004,0.000000e+00,2.329855e-06,5.636781e-07,0.000000e+00,0.000000e+00,...,0.000097,0.000087,0.000148,0.000423,0.000495,1.401582e-06,0.000174,0.001861,0.000543,0.000365
N098_C10,0.000131,0.000014,4.210477e-08,2.568290e-07,0.000005,0.000000e+00,1.860449e-06,4.496835e-07,0.000000e+00,0.000000e+00,...,0.000078,0.000093,0.000113,0.000312,0.000443,5.729589e-07,0.000126,0.001258,0.000465,0.000292
L027_A6,0.000263,0.000029,2.106031e-07,8.567068e-07,0.000004,0.000000e+00,9.377570e-07,0.000000e+00,3.005615e-06,0.000000e+00,...,0.000192,0.000248,0.000112,0.000392,0.000668,2.868886e-07,0.000302,0.005993,0.000802,0.000625
L072_A7,0.000367,0.000033,0.000000e+00,2.552480e-06,0.000018,3.797135e-07,9.283828e-07,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000218,0.000514,0.000207,0.000659,0.000878,2.092901e-06,0.000341,0.007661,0.000717,0.000677
L078_A8,0.000207,0.000025,0.000000e+00,2.719225e-06,0.000010,2.808028e-06,2.971054e-06,0.000000e+00,1.588838e-06,0.000000e+00,...,0.000303,0.000686,0.000164,0.000365,0.000617,4.954886e-07,0.000326,0.012547,0.000772,0.000435
L140_A9,0.000377,0.000106,2.925475e-07,2.573542e-06,0.000012,0.000000e+00,4.283263e-06,1.034071e-06,0.000000e+00,0.000000e+00,...,0.000194,0.000192,0.000227,0.000718,0.001031,1.854059e-06,0.000290,0.005113,0.001109,0.000708
L102_A10,0.000362,0.000031,0.000000e+00,5.556004e-07,0.000025,0.000000e+00,0.000000e+00,0.000000e+00,1.298975e-06,5.202356e-07,...,0.000185,0.000336,0.000175,0.000474,0.000576,1.240284e-06,0.000259,0.005516,0.000489,0.000520
L115_A11,0.000443,0.000016,0.000000e+00,2.420651e-07,0.000003,0.000000e+00,5.276134e-07,0.000000e+00,2.825895e-07,0.000000e+00,...,0.000109,0.000078,0.000122,0.000275,0.000304,0.000000e+00,0.000106,0.002741,0.000258,0.000237


In [38]:
T_labels_set = []
for i in list(range(0,14)):
    if i <4:
        T_labels_set.append('Control')
    else:
        T_labels_set.append('Active')
        
# Convert string column to integer
def str_column_to_int(labels):
    unique = set(labels)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for label in labels:
        label_int = lookup[label]
    return lookup

# convert class column to integers
str_column_to_int(T_labels_set)

T_Labels = []
for label in T_labels_set:
    int_ = str_column_to_int(T_labels_set)[label]
    T_Labels.append(int_)
T_Labels

[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [39]:
x_train, x_test, y_train, y_test = train_test_split(data_T_cell, T_Labels, test_size = 0.2, random_state = 1992)

In [40]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=1000,random_state = 42)
random_forest.fit(x_train, y_train)

Y_prediction = random_forest.predict(x_test)

random_forest.score(x_train, y_train)
acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
print(round(acc_random_forest,2,), "%")

100.0 %


In [41]:
# Test Random Forest
print('predict labels:',random_forest.predict(x_test))
print('True labels:',y_test)

predict labels: [0 1 1]
True labels: [0, 1, 1]


In [42]:
# select top 30 important features
importances = pd.DataFrame({'feature':x_train.columns,'importance':np.round(random_forest.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
T_important_gene_list_100 = importances.head(100).index.values.tolist()
T_important_gene_list_500 = importances.head(500).index.values.tolist()
importances.head(100)

,importance
feature,
ARHGAP18,0.003
CIPC,0.003
CCDC57,0.003
SOX30,0.003
ACBD4,0.002
KDELC2,0.002
CPM,0.002
AVL9,0.002
ALG14,0.002


In [53]:
t_cell_overlap_list = []
for a in data_T_cell_Diff_list:
    for b in T_important_gene_list_100:
        if a == b:
            t_cell_overlap_list.append(a)
t_cell_overlap_list

[]

------------------------------------------------------------

### Random forest for whole-blood-cell

In [43]:
data_Whole_Blood = data_Whole_Blood.drop(['WB_Control_Avg','WB_Active_Avg','WB_Active/Control','WB_Active-Control'],1)
data_Whole_Blood = data_Whole_Blood.transpose()
data_Whole_Blood

Tracking_ID,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
BRH704545_C1,0.000090,0.000051,0.0,0.000002,0.000011,0.000000e+00,0.000000e+00,0.000004,6.397107e-06,6.991717e-07,...,0.000033,0.000044,0.000017,0.000056,0.000544,0.000000e+00,0.000125,0.021955,0.000461,0.000067
BRH692283_C2,0.000087,0.000039,0.0,0.000004,0.000015,0.000000e+00,1.243204e-06,0.000000,6.638944e-07,1.595665e-06,...,0.000049,0.000033,0.000026,0.000070,0.000447,7.603287e-07,0.000097,0.019945,0.000405,0.000090
BRH712959_C3,0.000100,0.000057,0.0,0.000004,0.000008,3.623542e-07,1.282967e-06,0.000008,6.843582e-07,1.645282e-06,...,0.000042,0.000075,0.000023,0.000076,0.000602,0.000000e+00,0.000105,0.022112,0.000508,0.000081
BRH721340_C4,0.000060,0.000053,0.0,0.000005,0.000017,0.000000e+00,3.993951e-06,0.000006,2.607915e-05,1.279752e-06,...,0.000030,0.000032,0.000016,0.000047,0.000366,3.044846e-07,0.000060,0.020400,0.000376,0.000042
BRH721344_C5,0.000074,0.000043,0.0,0.000001,0.000009,0.000000e+00,5.477447e-06,0.000002,6.124665e-05,0.000000e+00,...,0.000038,0.000057,0.000014,0.000057,0.000381,5.558864e-07,0.000068,0.016158,0.000388,0.000077
SB09_A1,0.000072,0.000077,0.0,0.000003,0.000010,0.000000e+00,2.638443e-06,0.000000,2.107571e-06,2.534406e-06,...,0.000050,0.000098,0.000033,0.000082,0.000650,1.311152e-06,0.000124,0.020366,0.000558,0.000131
SB46_A2,0.000067,0.000054,0.0,0.000004,0.000018,0.000000e+00,1.731645e-06,0.000000,0.000000e+00,1.111248e-06,...,0.000046,0.000107,0.000027,0.000065,0.000502,0.000000e+00,0.000088,0.016497,0.000347,0.000088
SB60_A3,0.000028,0.000026,0.0,0.000005,0.000003,0.000000e+00,9.793529e-07,0.000002,2.082489e-06,1.252627e-06,...,0.000071,0.000307,0.000023,0.000045,0.000491,4.853172e-07,0.000102,0.016206,0.000408,0.000073
SB72_A4,0.000025,0.000025,0.0,0.000001,0.000004,0.000000e+00,0.000000e+00,0.000001,0.000000e+00,7.905754e-07,...,0.000048,0.000282,0.000013,0.000036,0.000396,4.154654e-06,0.000055,0.011677,0.000248,0.000053
SB79_A5,0.000094,0.000087,0.0,0.000002,0.000010,0.000000e+00,1.340557e-06,0.000003,1.430121e-06,8.595505e-07,...,0.000109,0.000364,0.000046,0.000100,0.000495,4.007764e-06,0.000110,0.008416,0.000415,0.000150


In [44]:
WB_labels_set = []
for i in list(range(0,10)):
    if i <5:
        WB_labels_set.append('Control')
    else:
        WB_labels_set.append('Active')
        
# Convert string column to integer
def str_column_to_int(labels):
    unique = set(labels)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for label in labels:
        label_int = lookup[label]
    return lookup

# convert class column to integers
str_column_to_int(WB_labels_set)

WB_Labels = []
for label in WB_labels_set:
    int_ = str_column_to_int(WB_labels_set)[label]
    WB_Labels.append(int_)
WB_Labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

In [45]:
x_train, x_test, y_train, y_test = train_test_split(data_Whole_Blood, WB_Labels, test_size = 0.2, random_state = 1992)

In [46]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=5000,random_state = 42)
random_forest.fit(x_train, y_train)

Y_prediction = random_forest.predict(x_test)

random_forest.score(x_train, y_train)
acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
print(round(acc_random_forest,2,), "%")

100.0 %


In [47]:
# Test Random Forest
print('predict labels:',random_forest.predict(x_test))
print('True labels:',y_test)

predict labels: [0 1]
True labels: [0, 1]


In [48]:
# select top 30 important features
importances = pd.DataFrame({'feature':x_train.columns,'importance':np.round(random_forest.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
WB_important_gene_list_100 = importances.head(100).index.values.tolist()
WB_important_gene_list_500 = importances.head(500).index.values.tolist()
importances.head(100)

,importance
feature,
HIST1H4H,0.002
C1orf226,0.001
DEPDC1B,0.001
C6orf62,0.001
GLDC,0.001
LINC01134,0.001
RPH3A,0.001
MAP3K13,0.001
DYRK1B,0.001


In [52]:
whole_blood_overlap_list = []
for a in data_Whole_Blood_Diff_list:
    for b in WB_important_gene_list_100:
        if a == b:
            whole_blood_overlap_list.append(a)
whole_blood_overlap_list

[]

### see overlap

In [49]:
overlap_gene_list = []
for t_gene in T_important_gene_list_500:
    for wb_gene in WB_important_gene_list_500:
        if t_gene == wb_gene:
            overlap_gene_list.append(t_gene)
overlap_gene_list       

['HACD3',
 'ARHGAP10',
 'SCARB2',
 'PRKACA',
 'ARL1',
 'ASUN',
 'PAPD5',
 'ERICH3',
 'MTX2',
 'HIBCH',
 'PAQR8',
 'PAQR3',
 'PAPSS1',
 'PA2G4',
 'MYDGF',
 'PACSIN3',
 'PSMA1',
 'CD82',
 'CTNNA1',
 'PRSS33',
 'PARD6A',
 'BTN2A2',
 'C4orf33',
 'SLC38A9']

### all gene distribution

In [10]:
data_NL = pd.read_csv("All_gene_NL.csv")
data_NL.head(10)

,tracking_id,N068,N048,N090,N098,L005,L101,LL062,L031,L027,L072,L078,L140,L102,L115,L133,L137,L149,L074
0,A1BG,4.963180,6.100440,4.358130,5.022150,6.284900,5.427030,2.708140,5.325740,4.582450,5.844510,3.290780,5.800160,6.988690,14.071100,1.629810,6.197470,1.700150,3.008110
1,A1BG-AS1,0.745182,0.364248,0.700179,0.540295,0.433740,0.449474,0.838054,0.496303,0.504396,0.519882,0.395447,1.635840,0.606268,0.510604,0.225992,0.596801,0.630004,0.388694
2,A1CF,0.000000,0.000000,0.008268,0.001614,0.000000,0.001856,0.001549,0.000000,0.003666,0.000000,0.000000,0.004506,0.000000,0.000000,0.009408,0.000000,0.000000,0.000000
3,A2M,0.009950,0.057608,0.006720,0.009844,0.096690,0.003774,0.006305,0.141589,0.014911,0.040649,0.043139,0.039638,0.010721,0.007690,0.101929,0.000000,0.033799,0.033222
4,A2M-AS1,0.480230,0.143089,0.112146,0.189595,0.111818,0.218373,0.231652,0.231841,0.074757,0.294029,0.152093,0.182653,0.477597,0.094097,0.099174,0.339683,0.261294,0.118479
5,A2ML1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006047,0.044547,0.000000,0.000000,0.000000,0.000000,0.000000,0.007622,0.004118
6,A2MP1,0.029018,0.103114,0.073276,0.071310,0.016845,0.032930,0.027552,0.000000,0.016322,0.014785,0.047134,0.065971,0.000000,0.016762,0.013873,0.016199,0.095305,0.029117
7,A3GALT2,0.000000,0.000000,0.017728,0.017236,0.040740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015927,0.000000,0.000000,0.000000,0.019574,0.049625,0.000000
8,A4GALT,0.000000,0.023710,0.000000,0.000000,0.000000,0.000000,0.022104,0.097337,0.052313,0.000000,0.025206,0.000000,0.025065,0.008978,0.014869,0.017388,0.000000,0.000000
9,A4GNT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010039,0.000000,0.000000,0.010436,0.000000,0.000000


In [11]:
data_NL_control = data_NL[['tracking_id','N068','N048','N090','N098']]
data_NL_active = data_NL[['tracking_id','L027','L072','L078','L140',
                          'L102','L115','L133','L137','L149','L074']]

In [12]:
data_NL_control.head()

,tracking_id,N068,N048,N090,N098
0,A1BG,4.963180,6.100440,4.358130,5.022150
1,A1BG-AS1,0.745182,0.364248,0.700179,0.540295
2,A1CF,0.000000,0.000000,0.008268,0.001614
3,A2M,0.009950,0.057608,0.006720,0.009844
4,A2M-AS1,0.480230,0.143089,0.112146,0.189595


In [24]:
data_NL_control['Control_Avg'] = (data_NL_control['N068']
                         +data_NL_control['N048']
                         +data_NL_control['N090']
                         +data_NL_control['N098'])/4

data_NL_active['Active_Avg'] = (data_NL_active['L027']
                         +data_NL_active['L072']
                         +data_NL_active['L078']
                         +data_NL_active['L140']
                         +data_NL_active['L102']
                         +data_NL_active['L115']
                         +data_NL_active['L133']
                         +data_NL_active['L137']
                         +data_NL_active['L149']
                         +data_NL_active['L074'])/10

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [25]:
data_NL_control_gene_avg = data_NL_control[['tracking_id','Control_Avg']]
data_NL_control_gene_avg
data_NL_active_gene_avg = data_NL_active[['tracking_id','Active_Avg']]

In [28]:
data_NL_control_gene_avg.head()

,tracking_id,Control_Avg
0,A1BG,5.110975
1,A1BG-AS1,0.587476
2,A1CF,0.002470
3,A2M,0.021031
4,A2M-AS1,0.231265
